In [1]:
import torch 
from torch.utils.data import DataLoader

from datasets import load_dataset
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

import numpy as np
import pandas as pd

import time
import random

import sys
sys.path.append("../")
from src.utils import predict_with_model
from src.model import BertSiameseModel

In [2]:
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Predict

In [ ]:
test_etl = pd.read_parquet("../dataset/test_data.parquet")
test_data = load_dataset("parquet", data_files="../preprocessed/test_pairs_preprocessed.parquet", split="train")

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2",
                                               truncation=True, return_tensors='pt', 
                                               model_max_length=256 )

test_texts = test_data.remove_columns( ['categories1', 'categories2'])

test_texts_1 = test_texts.map(lambda x : bert_tokenizer(x['attr_keys1'], x['attr_vals1'], truncation=True))
test_texts_1 = test_texts_1.remove_columns(['variantid1', 'variantid2'])
test_texts_1 = test_texts_1.remove_columns(['name_bert_642', 'attr_vals1', 'attr_keys1', 'attr_keys2', 'attr_vals2'])

test_texts_2 = test_texts.map(lambda x : bert_tokenizer(x['attr_keys2'], x['attr_vals2'], truncation=True))
test_texts_2 = test_texts_2.remove_columns(['variantid1', 'variantid2'])
test_texts_2 = test_texts_2.remove_columns(['name_bert_641', 'attr_vals1', 'attr_keys1', 'attr_keys2', 'attr_vals2'])

test_texts_2 = test_texts_2.rename_column("name_bert_642", "name_bert")
test_texts_1 = test_texts_1.rename_column("name_bert_641", "name_bert")

In [5]:
data_collator = DataCollatorWithPadding(bert_tokenizer)
test_text_dataloader = DataLoader(test_texts_1, batch_size=16, shuffle=False, collate_fn=data_collator)
test_text_dataloader2 = DataLoader(test_texts_2, batch_size=16, shuffle=False, collate_fn=data_collator)

In [6]:
test_pairs = pd.read_parquet("../dataset/test_pairs_wo_target.parquet")

In [7]:
siam = torch.load("../models/1685637489.722339.pth", map_location=torch.device('cpu'))
device = "cuda:0" if torch.cuda.is_available() else 'cpu'
predictions = predict_with_model(siam, test_text_dataloader, test_text_dataloader2, device)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [8]:
test_pairs["target"] = predictions

In [9]:
# Разные
for _, row in test_pairs[test_pairs['target'] < 0.5].head(15).iterrows():
  id1 = int(row['variantid1'])
  id2 = int(row['variantid2'])
  
  print(row['target'] , test_etl[test_etl['variantid'] == id1]['name'].to_list(), test_etl[test_etl['variantid'] == id2]['name'].to_list())

0.0030309027060866356 ['Батарейка AAA щелочная Perfeo LR03/10BL Super Alkaline 10 шт'] ['Батарейка AAA щелочная Perfeo LR03/2BL mini Super Alkaline 2 шт 2 упаковки']
0.044209353625774384 ['Смартфон Ulefone Armor X5 3/32 ГБ, черный, красный'] ['Смартфон Ulefone Armor X3 2/32 ГБ, черный, красный']
0.0022869708482176065 ['Цифровой кабель TV-COM HDMI 1.4 (M/ M) Full HD 1080p 3 м чёрный (CG150S-3M)'] ['Кабель  HDMI 1.4 (Male/Male) (CG150S-1.5M), черный + подарок']
0.0007595684728585184 ['Смартфон Vivo Y93 1815 3/32 ГБ, черный'] ['Смартфон Vivo Y81 3/32 ГБ, черный']
0.004558359272778034 ['Смартфон Blackview BV4900 3/32 ГБ, оранжевый'] ['Смартфон Blackview BV4900 3/32 ГБ, оранжевый, черный']
0.12988369166851044 ['Картридж лазерный Brother TN2275 черный (2600стр.) для Brother HL2240/2250/DCP7060/7070/MFC7630/7860'] ['Картридж Brother TN2275, черный, для лазерного принтера']
0.21709057688713074 ['Аккумулятор для Samsung NP300V 11.1V 4400mAh TopON'] ['Аккумулятор для Samsung NT300V 11.1V 4400mAh

In [10]:
# Похожие
for _, row in test_pairs[test_pairs['target'] > 0.5].head(15).iterrows():
  id1 = int(row['variantid1'])
  id2 = int(row['variantid2'])
  
  print(row['target'] , test_etl[test_etl['variantid'] == id1]['name'].to_list(), test_etl[test_etl['variantid'] == id2]['name'].to_list())

0.9166208505630493 ['Игровая мышь проводная A4Tech Bloody P93, 8 кнопок, подсветка, 5000 dpi, Bullet'] ['Мышь A4Tech Bloody P93s Bullet, серый, оптическая (8000dpi), USB (8 кнопок)']
0.8617610931396484 ['M42-M42 (17-31) M42 - M42 Крепление фокусировочного кольца геликоида для объектива '] ['Переходные кольца/адаптеры для объективов,M42-M42(17-31)']
0.9954909682273865 ['Дисплей для Xiaomi Redmi Note 4X в сборе с тачскрином и рамкой, черный'] ['Дисплей для Xiaomi Redmi Note 4X в сборе с тачскрином (белый) (у телефона отсутствуют винты снизу)']
0.9961600303649902 ['Ремешок силиконовый GSMIN Sport Band 20 для Huawei Watch GT Active (Дизайн 9)'] ['Ремешок силиконовый GSMIN Sport Band 20 для Huawei Watch GT Active (Дизайн 1)']
0.6777706742286682 ['Шлейф с вибромотором iPhone 7 Plus'] ['Шлейф материнской платы для iPhone 7 Plus']
0.995583713054657 ['Компьютерный корпус Exegate XP-330U-XP600, черный '] ['Корпус ATX Exegate XP-330U 500 Вт чёрный EX272730RUS']
0.9228420257568359 ['Сим лоток для 

In [11]:
test_pairs.to_csv(f"submit_{time.time()}.csv", index=False)